In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

Predictions (deep learning model)

In [16]:
emb_df = pd.read_csv("sbert_embeddings.csv", header=None) # embeddings
data_df = pd.read_json("dataset.json") # targets
print(emb_df.shape, data_df.shape)

(148122, 1536) (148122, 6)


In [17]:
X = np.array(emb_df.values.tolist())[:]
Y = pd.get_dummies(data_df['category']).values[:]  # One-hot encoding of categories
print(X.shape, Y.shape)

(148122, 1536) (148122, 15)


In [34]:
# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(Y.shape[1], activation='softmax')  # Prediction head
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(X_test)
predicted_categories = np.argmax(predictions, axis=1)


Epoch 1/10
25/25 [==============================] - 2s 17ms/step - loss: 1.1761 - accuracy: 0.6687
Epoch 2/10
25/25 [==============================] - 0s 17ms/step - loss: 0.6002 - accuracy: 0.8238
Epoch 3/10
25/25 [==============================] - 1s 21ms/step - loss: 0.4155 - accuracy: 0.8963
Epoch 4/10
25/25 [==============================] - 0s 17ms/step - loss: 0.3198 - accuracy: 0.9075
Epoch 5/10
25/25 [==============================] - 0s 17ms/step - loss: 0.2277 - accuracy: 0.9337
Epoch 6/10
25/25 [==============================] - 0s 17ms/step - loss: 0.1626 - accuracy: 0.9463
Epoch 7/10
25/25 [==============================] - 0s 17ms/step - loss: 0.1237 - accuracy: 0.9700
Epoch 8/10
25/25 [==============================] - 1s 20ms/step - loss: 0.1141 - accuracy: 0.9675
Epoch 9/10
25/25 [==============================] - 0s 18ms/step - loss: 0.0900 - accuracy: 0.9750
Epoch 10/10
7/7 [==============================] - 0s 5ms/step


In [6]:
N = len(predicted_categories)
c = 0
for pred, y in zip(predicted_categories, np.argmax(Y_test, axis=1)):
    if pred == y:
        c += 1
print(c, N)
print("ACC", c/N)

1712 2000
ACC 0.856


# Preliminary hyperparameter tuning

In [ ]:
X_train_global, X_test_global, Y_train_global, Y_test_global = train_test_split(X, Y, test_size=0.2, random_state=42)

In [101]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier 
from sklearn.model_selection import StratifiedKFold, cross_val_score
from kerastuner.tuners import Hyperband

# Define the model creation function with hyperparameters
def create_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=256, max_value=1024, step=128),
                    activation='relu', kernel_regularizer=regularizers.l2(hp.Float('weight_decay', min_value=1e-6, max_value=1e-2, sampling='log'))))
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=Y.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize KerasClassifier with the model creation function
model = KerasClassifier(build_fn=create_model, batch_size=32, verbose=0)

# Perform hyperparameter tuning using Hyperband tuner
tuner = Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='hyperband',
    project_name='my_tuning_project'
)

tuner.search_space_summary()

# Run the hyperparameter search
tuner.search(X_train_global, Y_train_global, epochs=10, batch_size=32, validation_split=0.3, callbacks=[EarlyStopping('val_loss', patience=5)])

# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_hp.values)

# Retrain the model with the best hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
# Ensure correct dimensions
best_model.fit(X[0], Y[0], epochs=1, batch_size=32, verbose=False)
best_model.save("data/best_model")

Reloading Tuner from hyperband/my_tuning_project/tuner0.json
Search space summary
Default search space size: 4
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 256, 'max_value': 1024, 'step': 128, 'sampling': 'linear'}
weight_decay (Float)
{'default': 1e-06, 'conditions': [], 'min_value': 1e-06, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
dropout (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
Best Hyperparameters:
{'units_1': 896, 'weight_decay': 4.302778030652062e-05, 'dropout': 0.30000000000000004, 'learning_rate': 0.0004055961741658466, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
INFO:tensorflow:Assets written to: data/best_model/assets


INFO:tensorflow:Assets written to: data/best_model/assets


# Cross validation

The last batch is important - it represents the batch not used when hypertuning - hyperparams can also overfit!!

In [103]:
## HYPER CROSS VALIDATION

import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


num_folds = 3
random_state = 42

# Initialize StratifiedKFold
stratkf = StratifiedKFold(n_splits=num_folds, shuffle=False, random_state=random_state)

# Initialize lists to store metrics for each iteration
accuracies, precisions, recalls, f1_scores = [], [], [], []

# Loop over folds
for fold, (train_indices, test_indices) in enumerate(stratkf.split(X_train_global, np.argmax(Y_train_global, axis=1))):
    print(f"Training on fold {fold + 1}...")

    # Split the data for this fold
    X_train, X_test = X[train_indices], X[test_indices]
    Y_train, Y_test = Y[train_indices], Y[test_indices]

    model = tf.keras.models.load_model("data/best_model")
    model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=True)

    # Train the model
    # model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=True)

    # Make predictions on the test set
    predictions = model.predict(X_test)
    predicted_categories = np.argmax(predictions, axis=1)

    # Calculate metrics for this iteration
    accuracy = accuracy_score(np.argmax(Y_test, axis=1), predicted_categories)
    precision = precision_score(np.argmax(Y_test, axis=1), predicted_categories, average='weighted')
    recall = recall_score(np.argmax(Y_test, axis=1), predicted_categories, average='weighted')
    f1 = f1_score(np.argmax(Y_test, axis=1), predicted_categories, average='weighted')

    # Append metrics to the lists
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    # Display metrics for this iteration
    print(f"Accuracy for fold {fold + 1}: {accuracy:.4f}")
    print(f"Precision for fold {fold + 1}: {precision:.4f}")
    print(f"Recall for fold {fold + 1}: {recall:.4f}")
    print(f"F1 Score for fold {fold + 1}: {f1:.4f}\n")

# Calculate and output the average metrics
average_accuracy = np.mean(accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)

print(f"\nAverage Accuracy: {average_accuracy:.4f}")
print(f"Average Precision: {average_precision:.4f}")
print(f"Average Recall: {average_recall:.4f}")
print(f"Average F1 Score: {average_f1:.4f}")

Training on fold 1...
Epoch 1/10
4629/4629 [==============================] - 138s 30ms/step - loss: 0.7753 - accuracy: 0.7673
Epoch 2/10
4629/4629 [==============================] - 136s 29ms/step - loss: 0.7298 - accuracy: 0.7822
Epoch 3/10
4629/4629 [==============================] - 131s 28ms/step - loss: 0.7039 - accuracy: 0.7920
Epoch 4/10
4629/4629 [==============================] - 136s 29ms/step - loss: 0.6853 - accuracy: 0.7980
Epoch 5/10
4629/4629 [==============================] - 140s 30ms/step - loss: 0.6677 - accuracy: 0.8049
Epoch 6/10
4629/4629 [==============================] - 138s 30ms/step - loss: 0.6557 - accuracy: 0.8094
Epoch 7/10
4629/4629 [==============================] - 151s 33ms/step - loss: 0.6444 - accuracy: 0.8132
Epoch 8/10
4629/4629 [==============================] - 242s 52ms/step - loss: 0.6336 - accuracy: 0.8165
Epoch 9/10
4629/4629 [==============================] - 156s 34ms/step - loss: 0.6264 - accuracy: 0.8197
Epoch 10/10
4629/4629 [==========

# Global testing

In [19]:
model = tf.keras.models.load_model("data/best_model")
model.fit(X_train_global, Y_train_global, epochs=10, batch_size=32, verbose=True)
# Train the model
# model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=True)

# Make predictions on the test set
predictions = model.predict(X_test_global)
predicted_categories = np.argmax(predictions, axis=1)

# Calculate metrics for this iteration
accuracy = accuracy_score(np.argmax(Y_test_global, axis=1), predicted_categories)
precision = precision_score(np.argmax(Y_test_global, axis=1), predicted_categories, average='weighted')
recall = recall_score(np.argmax(Y_test_global, axis=1), predicted_categories, average='weighted')
f1 = f1_score(np.argmax(Y_test_global, axis=1), predicted_categories, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}\n")

Epoch 1/10
4166/4166 [==============================] - 141s 33ms/step - loss: 0.7754 - accuracy: 0.7671
Epoch 2/10
4166/4166 [==============================] - 139s 33ms/step - loss: 0.7302 - accuracy: 0.7815
Epoch 3/10
4166/4166 [==============================] - 137s 33ms/step - loss: 0.7012 - accuracy: 0.7922
Epoch 4/10
4166/4166 [==============================] - 141s 34ms/step - loss: 0.6815 - accuracy: 0.7993
Epoch 5/10
4166/4166 [==============================] - 141s 34ms/step - loss: 0.6655 - accuracy: 0.8054
Epoch 6/10
4166/4166 [==============================] - 143s 34ms/step - loss: 0.6510 - accuracy: 0.8100
Epoch 7/10
4166/4166 [==============================] - 148s 36ms/step - loss: 0.6348 - accuracy: 0.8160
Epoch 8/10
4166/4166 [==============================] - 148s 36ms/step - loss: 0.6245 - accuracy: 0.8189
Epoch 9/10
4166/4166 [==============================] - 186s 45ms/step - loss: 0.6146 - accuracy: 0.8225
Epoch 10/10
463/463 [==============================] - 

# Backup

In [95]:
## BACKUP CROSS VALIDATION

import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras import regularizers

# Assuming 'X' is your feature matrix and 'Y' is the target variable

# Define hyperparameters
num_folds = 5
epochs = 10
batch_size = 32
learning_rate = 0.001
weight_decay = 1e-5
random_state = 42

# Initialize StratifiedKFold
stratkf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=random_state)

# Initialize lists to store metrics for each iteration
accuracies, precisions, recalls, f1_scores = [], [], [], []

# Define your TensorFlow model with hyperparameters
def create_model(input_dim, output_dim, learning_rate, weight_decay):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(output_dim, activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Loop over folds
for fold, (train_indices, test_indices) in enumerate(stratkf.split(X, np.argmax(Y, axis=1))):
    print(f"Training on fold {fold + 1}...")

    # Split the data for this fold
    X_train, X_test = X[train_indices], X[test_indices]
    Y_train, Y_test = Y[train_indices], Y[test_indices]

    # Create and compile the model
    model = create_model(X_train.shape[1], Y_train.shape[1], learning_rate, weight_decay)

    # Train the model
    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=True)

    # Make predictions on the test set
    predictions = model.predict(X_test)
    predicted_categories = np.argmax(predictions, axis=1)

    # Calculate metrics for this iteration
    accuracy = accuracy_score(np.argmax(Y_test, axis=1), predicted_categories)
    precision = precision_score(np.argmax(Y_test, axis=1), predicted_categories, average='weighted')
    recall = recall_score(np.argmax(Y_test, axis=1), predicted_categories, average='weighted')
    f1 = f1_score(np.argmax(Y_test, axis=1), predicted_categories, average='weighted')

    # Append metrics to the lists
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    # Display metrics for this iteration
    print(f"Accuracy for fold {fold + 1}: {accuracy:.4f}")
    print(f"Precision for fold {fold + 1}: {precision:.4f}")
    print(f"Recall for fold {fold + 1}: {recall:.4f}")
    print(f"F1 Score for fold {fold + 1}: {f1:.4f}\n")

# Calculate and output the average metrics
average_accuracy = np.mean(accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)

print(f"\nAverage Accuracy: {average_accuracy:.4f}")
print(f"Average Precision: {average_precision:.4f}")
print(f"Average Recall: {average_recall:.4f}")
print(f"Average F1 Score: {average_f1:.4f}")

/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Training on fold 1...
Epoch 1/10
25/25 [==============================] - 2s 20ms/step - loss: 1.1784 - accuracy: 0.6750
Epoch 2/10
25/25 [==============================] - 0s 18ms/step - loss: 0.5984 - accuracy: 0.8425
Epoch 3/10
25/25 [==============================] - 0s 19ms/step - loss: 0.4341 - accuracy: 0.8712
Epoch 4/10
25/25 [==============================] - 0s 19ms/step - loss: 0.3204 - accuracy: 0.9187
Epoch 5/10
25/25 [==============================] - 0s 19ms/step - loss: 0.2659 - accuracy: 0.9262
Epoch 6/10
25/25 [==============================] - 0s 19ms/step - loss: 0.1913 - accuracy: 0.9500
Epoch 7/10
25/25 [==============================] - 1s 21ms/step - loss: 0.1294 - accuracy: 0.9675
Epoch 8/10
25/25 [==============================] - 1s 22ms/step - loss: 0.1187 - accuracy: 0.9712
Epoch 9/10
25/25 [==============================] - 0s 19ms/step - loss: 0.1136 - accuracy: 0.9762
Epoch 10/10
7/7 [==============================] - 0s 5ms/step
Accuracy for fold 1: 0.8

/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


25/25 [==============================] - 2s 21ms/step - loss: 1.1781 - accuracy: 0.6737
Epoch 2/10
25/25 [==============================] - 1s 25ms/step - loss: 0.6211 - accuracy: 0.8300
Epoch 3/10
25/25 [==============================] - 1s 24ms/step - loss: 0.4279 - accuracy: 0.8788
Epoch 4/10
25/25 [==============================] - 0s 19ms/step - loss: 0.3411 - accuracy: 0.9038
Epoch 5/10
25/25 [==============================] - 0s 19ms/step - loss: 0.2299 - accuracy: 0.9350
Epoch 6/10
25/25 [==============================] - 0s 19ms/step - loss: 0.1917 - accuracy: 0.9525
Epoch 7/10
25/25 [==============================] - 0s 19ms/step - loss: 0.1554 - accuracy: 0.9650
Epoch 8/10
25/25 [==============================] - 0s 19ms/step - loss: 0.1536 - accuracy: 0.9588
Epoch 9/10
25/25 [==============================] - 0s 19ms/step - loss: 0.1001 - accuracy: 0.9750
Epoch 10/10
7/7 [==============================] - 0s 4ms/step
Accuracy for fold 2: 0.9150
Precision for fold 2: 0.9154


/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


25/25 [==============================] - 2s 23ms/step - loss: 1.2117 - accuracy: 0.6513
Epoch 2/10
25/25 [==============================] - 0s 17ms/step - loss: 0.6579 - accuracy: 0.8225
Epoch 3/10
25/25 [==============================] - 0s 15ms/step - loss: 0.4806 - accuracy: 0.8725
Epoch 4/10
25/25 [==============================] - 0s 15ms/step - loss: 0.3543 - accuracy: 0.9162
Epoch 5/10
25/25 [==============================] - 0s 17ms/step - loss: 0.2892 - accuracy: 0.9312
Epoch 6/10
25/25 [==============================] - 1s 23ms/step - loss: 0.2085 - accuracy: 0.9350
Epoch 7/10
25/25 [==============================] - 0s 15ms/step - loss: 0.1961 - accuracy: 0.9500
Epoch 8/10
25/25 [==============================] - 0s 15ms/step - loss: 0.1195 - accuracy: 0.9725
Epoch 9/10
25/25 [==============================] - 0s 16ms/step - loss: 0.0934 - accuracy: 0.9762
Epoch 10/10
7/7 [==============================] - 0s 3ms/step
Accuracy for fold 4: 0.8950
Precision for fold 4: 0.8965


/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


25/25 [==============================] - 1s 15ms/step - loss: 1.1775 - accuracy: 0.6550
Epoch 2/10
25/25 [==============================] - 0s 15ms/step - loss: 0.5983 - accuracy: 0.8450
Epoch 3/10
25/25 [==============================] - 0s 15ms/step - loss: 0.4333 - accuracy: 0.8838
Epoch 4/10
25/25 [==============================] - 0s 16ms/step - loss: 0.2833 - accuracy: 0.9287
Epoch 5/10
25/25 [==============================] - 0s 16ms/step - loss: 0.2256 - accuracy: 0.9350
Epoch 6/10
25/25 [==============================] - 0s 14ms/step - loss: 0.1854 - accuracy: 0.9538
Epoch 7/10
25/25 [==============================] - 0s 15ms/step - loss: 0.1245 - accuracy: 0.9663
Epoch 8/10
25/25 [==============================] - 0s 15ms/step - loss: 0.0896 - accuracy: 0.9775
Epoch 9/10
25/25 [==============================] - 0s 15ms/step - loss: 0.0752 - accuracy: 0.9850
Epoch 10/10
7/7 [==============================] - 0s 3ms/step
Accuracy for fold 5: 0.8800
Precision for fold 5: 0.8596


/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
